## Modèle en production

Laurent Cetinsoy - Datadidacte

Une des supposition centrale pour qu'un modèle de machine learning marche est que la distribution des données ne diffère pas de celle des données d'entrainement.

On ne peut garantir la généralisation d'un modèle que si la distribution des données est similaire à celle de la distribution $ X_{prod} \tilde{} \,  P_{train}$

Ainsi, si les données que le modèle voient en production n'ont pas la même distribution (ne ressemblent pas) aux données de train, alors le modèle aura peu de chance de faire de bonne prédictions.

Il est donc important de surveille les données vues par le modèle en production.

Pour cela on va mesurer ce qu'on appelle le Data drift : on va mesurer à quel point les données s'écartent des données de train.

Et on pourra ainsi lever une alerte si c'est le cas.

## Utilisation Eurybia



En consultant la documentation de Eurybia (https://eurybia.readthedocs.io/en/latest/overview.html), expliquer le principe de fonctionnement de Euribya :

- A quoi sert le modèle de classification ?
- A-t-on besoin d’avoir les labels issus de la production pour pouvoir utiliser cette approche ?
- Quel est le critère pour déterminer qu’il y a un data-drift ?


**Eurybia met à disposition un modèle de classification binaire de données, spécifiant si la donnée fournie est une donnée d'entrainement ou une donnée réelle de production.**
- Pour faire simple, on donne un item au modèle, et celui-ci tente de déterminer si l'item appartient à X_train ou X_test.
- Les labels ne sont absolument pas nécessaires, puisqu'on ne s'intéresse pas à analyser la donnée de façon à obtenir le meilleur score possible, mais davantage à déterminer à quel point les données de train et de tests sont similaires. Ainsi, les labels nécessaires sont juste l'origine de l'item fourni.
- L'unité de mesure utilisée par Eurybia est l'AUC (Area Under roc Curve).
- Un AUC de 0.5 signifie que les données sont bien réparties autour de la courbe (au dessus et en dessous) et donc que les données de train et de production sont assez similaires.
- En revanche, un AUC de 1 démontre un décalage entre les deux sets de données, et donc qu'un drift des données a lieu.

Installer eurybia

In [ ]:
from eurybia import SmartDrift
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import joblib
import numpy as np


df = pd.read_csv('houses.csv')
X = df[['size', 'nb_rooms', 'garden']]
y = df['price']
X_train, X_prod, y_train, y_prod = train_test_split(X, y, train_size=0.4)
model = LinearRegression()
model.fit(X, y)
joblib.dump(model, "regression.joblib")

Utiliser eurybia pour monitorer la distribution des données. Dans un premier temps faire en sorte que les données de prod (df_current) soient de la même distribution que vos données d’entraînement. Pour cela vous pouvez split le dataset en deux et décider que l'un est X_train et l'autre est X_prod. Vérifier que Eurybia pense que le modèle ne drift pas. Attention à ne pas inclure le label (qui serait ici price)



In [ ]:
def evaluate(current, baseline) :

    sd = SmartDrift(
      df_current=current,
      df_baseline=baseline,
      dataset_names={"df_current": "Houses.csv", "df_baseline": "Test Houses.csv"} # Optional: Names for outputs
      )
    
    sd.compile(
      full_validation=True, # Optional: to save time, leave the default False value. If True, analyze consistency on modalities between columns.
      )
    
    sd.generate_report(
      output_file='output/report.html',
      title_story="Eurybia starts",
      title_description="Not sus at all", # Optional: add a subtitle to describe report
      )
    
    print("Done!")

evaluate(X_prod, X_train)

Faire en sorte d'introduire un drift dans vos données. Par exemple (méthode assez bourine) ajouter +1 à la colonne nb_rooms. Relancer eurybia et vérifier que la performance du modèle est bonne et qu'il y a donc un datadrift

In [ ]:
scale = 10

def addNoise(df, scale = 1.0) :
    rng = np.random.default_rng(42)
    noisy = df.copy()
        
    for col in noisy.select_dtypes(include=[np.number]).columns:
        std = noisy[col].std()
        noise = rng.normal(0, scale * std, size=len(noisy))
        noisy[col] += noise
    return noisy

X_prod_noisy = addNoise(X_prod, scale)
evaluate(X_prod_noisy, X_train)

Faire en sorte de faire un drift plus subtile mais remarquable. Eurybia est-il détectable avec Eurybia ?

In [ ]:
scale = 2

def addNoise(df, scale = 1.0) :
    rng = np.random.default_rng(42)
    noisy = df.copy()
        
    for col in noisy.select_dtypes(include=[np.number]).columns:
        std = noisy[col].std()
        noise = rng.normal(0, scale * std, size=len(noisy))
        noisy[col] += noise
    return noisy

X_prod_noisy = addNoise(X_prod, scale)
evaluate(X_prod_noisy, X_train)

**On remarque que l'AUC est à 0.64, ce qui est proche de 0.5, mais toutefois notable.** 

## Alibaba detect

Dans cette partie on va utiliser la librairie https://github.com/SeldonIO/alibi-detect pour faire la détection de problèmes
Installer la librairie avec pip

In [ ]:
from alibi_detect.od import OutlierVAE
from alibi_detect.saving import save_detector, load_detector
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import keras

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
tf.config.list_physical_devices('GPU')

Charger le jeu de donnée cifar10 avec keras et récupérer le train et le test puis,

Normaliser les données de train en faisant un MinMaxScaling (diviser par 255)

In [ ]:
m = max(np.max(x_train), np.max(x_test))
X_train_scaled = x_train / m
X_test_scaled = x_test / m

Dans le sous package alibbi_detect.datasets importer les  fonction fetch_cifar10c et corruption_types_cifar10c

In [ ]:
from alibi_detect.datasets import fetch_cifar10c, corruption_types_cifar10c

Afficher les types de corruption de données disponible avec la fonction corruption_types_cifar10c

In [ ]:
print(corruption_types_cifar10c())

Avec la fonction fetch_cifar10c récupérer des exemples corrompus de donnée ressemblant à cifar 10 et les stocker dans des variable X_corrupted et y_corrupted.

Vous choisirez 1 ou 2 corruptions parmis les suivantes : ['gaussian_noise', 'motion_blur', 'brightness', 'pixelate']

Vous spécifirez les argument severity=5 et return_X_y=True
attention le dataset peut être lourd si vous utilisez bcp de corruptions

In [ ]:
X_corrupted, y_corrupted = fetch_cifar10c(['gaussian_noise', 'brightness'], severity=5, return_X_y=True)

Normaliser les images corrompues

In [ ]:
X_corrupted_scaled = X_corrupted / m

Afficher plusieurs des images corrompues.

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 4, figsize=(8, 6))
axes = axes.ravel()
for i in range(12):
    axes[i].imshow(X_corrupted_scaled[i], cmap="gray")
    axes[i].axis("off")  # hide axes/ticks

plt.tight_layout()
plt.show()

On va maintenant prendre un modèle entraîné sur cifar10 pour voir l'impact des performances sur le modèle.

Avec la fonction  fetch_tf_model du module alibi_detect.utils.fetching, charger le modèle préentraîné resnet32 sur cifar10


In [ ]:
from alibi_detect.utils.fetching import fetch_tf_model, fetch_detector
dataset = 'cifar10'
model_name = 'resnet32'
model = fetch_tf_model(dataset, model_name)

Calculer la performance du model sur le jeu de train et de test

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train_scaled, y_train, verbose=1)
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=1)

print(f"Train loss: {train_loss:.4f}")
print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

Calculer la performance du modèle sur le jeu de donnée corrompu. Vous devriez observer qu'il chute significativement

In [ ]:
corr_loss, corr_accuracy = model.evaluate(X_corrupted_scaled, y_corrupted, verbose=1)

print(f"Corrupted loss: {corr_loss:.4f}")
print(f"Corrupted accuracy: {corr_accuracy:.4f}")

On va maintenant voir comment détecter les changement de distributions de données.

Pour les données non tabulaire ou à haute dimension on procéde généralement en deux étapes :

1. Faire une réduction de dimension
2. Faire un test permettant de voir si les données projetées ont changé de distribution ou pas

Il existe plusieurs manières de faire de la réduction de dimension. La plus classique est la PCA.

Il est possible également d'utiliser des Auto-encoder

Le code suivant permet de créer la première partie (l'encoder) d'un auto-encoder simple qui nous servira à réduire les dimension des données.

In [ ]:
from functools import partial
from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer, Reshape
from alibi_detect.cd.tensorflow import preprocess_drift

tf.random.set_seed(0)

# define encoder
encoding_dim = 32
encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(32, 32, 3)),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ]
)

Le drift detector a besoin d'une donnée de référence afin d'effectuer la comparaison avec les données à monitorer.
Créer une variable X_ref avec un échantillon aléatoire des données de test

In [ ]:
idx = np.random.choice(np.arange(X_test_scaled.shape[0]), int(X_test_scaled.shape[0] * 0.3))
X_ref = X_test_scaled[idx]
y_ref = y_test[idx]

A quoi sert le test statistique kolmogorov smirnoff ?

**Le test de kolmogorov smirnoff sert à déterminer si un échantillon de données suit une loi de probabilité particulière, connue et dont la fonction de répartition est déterminée.**
- Par exemple, étant donné un échantillon de données généré d'une façon inconnue, effectuer le test avec cet échantillon et une loi connue (par exemple la loi normale) permet d'affirmer ou réfuter le fait que l'échantillon inconnu soit issu de cette loi.

Instancier la classe KSDrift dans une variable nommée **detector**

Il faut lui passer le dataset de reference, une p value (prendre 0.05) et une fonction permettant de faire le preprocessing. On a créé la fonction pour vous


In [ ]:
from alibi_detect.cd.tensorflow import preprocess_drift
preprocess_function = partial(preprocess_drift, model=encoder_net, batch_size=32)

In [ ]:
from alibi_detect.cd import KSDrift

detector = KSDrift(X_ref, p_val=0.05, preprocess_fn=preprocess_function)

A laide du Drift detector et la méthode predict faire des prediction sur les données de test et sur les données corrompue pour voir si il détecte un changement de distribution

In [ ]:
r = detector.predict(X_test_scaled, drift_type='batch', return_p_val=True, return_distance=True)
print(f"Drift detected for test data: {True if r['data']['is_drift'] == 1 else False}")

In [ ]:
r = detector.predict(X_corrupted_scaled, drift_type='batch', return_p_val=True, return_distance=True)
print(f"Drift detected for corrupted data: {True if r['data']['is_drift'] == 1 else False}")